<a href="https://colab.research.google.com/github/jrctechmind/machine_learning/blob/main/Filtragem_Colaborativa_por_Usu%C3%A1rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Dados de exemplo: matriz de avaliações de usuários para itens
avaliacoes = np.array([
    [5, 3, 0, 0],
    [4, 0, 4, 1],
    [1, 1, 0, 5],
    [0, 0, 4, 4],
    [0, 1, 5, 4],
])

def similaridade_usuario(usuario1, usuario2):
    """
    Calcula a similaridade entre dois usuários usando a correlação de Pearson.
    Quanto maior o valor retornado, mais similaridade entre os usuários.

    Argumentos:
    usuario1 -- índice do primeiro usuário
    usuario2 -- índice do segundo usuário

    Retorna:
    similaridade -- valor de similaridade entre os usuários
    """
    avaliacoes_usuario1 = avaliacoes[usuario1]
    avaliacoes_usuario2 = avaliacoes[usuario2]
    avaliacoes_comuns = np.logical_and(avaliacoes_usuario1 != 0, avaliacoes_usuario2 != 0)

    if np.sum(avaliacoes_comuns) == 0:
        return 0

    avaliacoes_usuario1_comuns = avaliacoes_usuario1[avaliacoes_comuns]
    avaliacoes_usuario2_comuns = avaliacoes_usuario2[avaliacoes_comuns]

    media_usuario1 = np.mean(avaliacoes_usuario1_comuns)
    media_usuario2 = np.mean(avaliacoes_usuario2_comuns)

    numerador = np.sum((avaliacoes_usuario1_comuns - media_usuario1) * (avaliacoes_usuario2_comuns - media_usuario2))
    denominador = np.sqrt(np.sum((avaliacoes_usuario1_comuns - media_usuario1)**2)) * np.sqrt(np.sum((avaliacoes_usuario2_comuns - media_usuario2)**2))

    if denominador == 0:
        return 0

    similaridade = numerador / denominador
    return similaridade

def recomendar(usuario_alvo, k=2):
    """
    Faz recomendações para um usuário com base em usuários semelhantes.

    Argumentos:
    usuario_alvo -- índice do usuário para o qual desejamos fazer recomendações
    k -- número de usuários mais similares a considerar para recomendações (padrão: 2)

    Retorna:
    recomendacoes -- lista de índices dos itens recomendados
    """
    similaridades = [similaridade_usuario(usuario_alvo, outro_usuario) for outro_usuario in range(len(avaliacoes))]
    indices_similares = np.argsort(similaridades)[::-1][:k]  # Índices dos usuários mais similares

    predicoes = np.zeros(avaliacoes.shape[1])
    for indice_item in range(avaliacoes.shape[1]):
        if avaliacoes[usuario_alvo][indice_item] == 0:  # Verificar apenas itens não avaliados pelo usuário
            soma_similaridades = 0
            soma_avaliacoes_ponderadas = 0
            for usuario_similar in indices_similares:
                similaridade = similaridades[usuario_similar]
                avaliacao_usuario_similar = avaliacoes[usuario_similar][indice_item]
                soma_similaridades += np.abs(similaridade)
                soma_avaliacoes_ponderadas += similaridade * avaliacao_usuario_similar
            if soma_similaridades != 0:
                predicoes[indice_item] = soma_avaliacoes_ponderadas / soma_similaridades

    recomendacoes = np.argsort(predicoes)[::-1][:k]  # Índices dos itens recomendados com base nas predições
    return recomendacoes

# Exibição de instruções e interações com o usuário
print("Bem-vindo ao Sistema de Recomendação!")
print("Por favor, insira o número do usuário para o qual deseja fazer recomendações (0 a", len(avaliacoes) - 1, "):")
usuario_alvo = int(input())

if usuario_alvo < 0 or usuario_alvo >= len(avaliacoes):
    print("Número de usuário inválido. Por favor, insira um número válido.")
else:
    print("\nUsuário alvo selecionado:", usuario_alvo)
    print("Calculando a similaridade entre o usuário alvo e os demais usuários...")

    similaridades = [similaridade_usuario(usuario_alvo, outro_usuario) for outro_usuario in range(len(avaliacoes))]
    print("Similaridades calculadas com sucesso.")
    print("Usuários mais similares ao usuário alvo:")
    for i, similaridade in enumerate(similaridades):
        print("Usuário", i, "- Similaridade:", similaridade)

    print("\nFazendo recomendações para o usuário...")
    recomendacoes = recomendar(usuario_alvo)
    print("\nRecomendações para o usuário selecionado:")
    for i, item in enumerate(recomendacoes):
        print("  - Item", item)
        print("    Avaliação prevista:", avaliacoes[usuario_alvo][item])
        print("    Similaridade do usuário que deu a avaliação:", similaridades[item])

print("\nObrigado por usar nosso sistema de recomendação!")


Bem-vindo ao Sistema de Recomendação!
Por favor, insira o número do usuário para o qual deseja fazer recomendações (0 a 4 ):
1

Usuário alvo selecionado: 1
Calculando a similaridade entre o usuário alvo e os demais usuários...
Similaridades calculadas com sucesso.
Usuários mais similares ao usuário alvo:
Usuário 0 - Similaridade: 0
Usuário 1 - Similaridade: 1.0000000000000002
Usuário 2 - Similaridade: -1.0
Usuário 3 - Similaridade: 0
Usuário 4 - Similaridade: 1.0

Fazendo recomendações para o usuário...

Recomendações para o usuário selecionado:
  - Item 1
    Avaliação prevista: 0
    Similaridade do usuário que deu a avaliação: 1.0000000000000002
  - Item 3
    Avaliação prevista: 1
    Similaridade do usuário que deu a avaliação: 0

Obrigado por usar nosso sistema de recomendação!
